## Airline Delay Analysis with Jupyter Notebook

### Introduction 

In order to execute our data analysis in python, various libraries are needed to assist in performing operations as well as for data visualization through graphs. Next, we will assign out cloud connection variables and utilize the nzpy library to create a connection to our data source. 

In [ ]:
# import python libraries 
!pip install nzpy
!pip install pandasql
import numpy as np 
import nzpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
from pandasql import sqldf

In [ ]:
#Netezza Cloud Connection Information (you can use this to store credentials or run locally)
nz_host             = "nz-4279e2a3-0345-44f0-92e9-992d72d04198.eastus2.data-warehouse.cloud.ibm.com"
nz_port             = 5480
nz_database         = "demo_U1100008XCY"
nz_user             = "U1100008XCY"
nz_password         = "U1100008XCY!Password"


In [ ]:
nzcon = nzpy.connect(user=nz_user, password=nz_password, host=nz_host, database=nz_database, port=nz_port)


We have already completed the data loading portion of this demo. The first query in this notebook is to check that our tables are defined and available. 

In [ ]:
q = """
select tablename from _v_table
"""

df = pd.read_sql_query(q, nzcon)
#df.columns = df.columns.str
df

#### Quick View of Our Data 
The next query is a snapshot of the first couple of rows in our AIRLINE_DELAY_CAUSE_CURRENT. Becasue our first objective in this analysis is to assess the current state of Airline delays, we will want to use the recent data between 2019 and 2022 first. Here we can see sample rows for each of our columms. 

In [ ]:
q = """
SELECT 
*
FROM AIRLINE_DELAY_CAUSE_CURRENT
LIMIT 5
"""

df = pd.read_sql_query(q, nzcon)
pd.set_option('display.max_columns', None)
#df.columns = df.columns.str.decode('utf-8')
df

### Defining Colummns 

- Year 
- Month 
- Carrier: Abbreviation
- Carrier_Name: Carrier name full
- Airport: Abbreviation 
- Airport_City
- Airport_State: State Abbreviation 
- Airport_Name: Airport Name Full 
- Arr_Flights: Total Arrived Flights 
- Arr_Del15: Total Delayed Flights as defined by 15 minutes or more 
- Carrier_CT: Total delayed flights due to air carrier issues
- Weather_CT: Total delayed flights due to weather
- NAS_CT: Total delayed flights due to National Aviation System 
- Security_CT: Count of delayed flights caused by security 
- Late_Aircraft_CT: Count of delayed flights caused by late arriving aircraft 
- Arr_Cancelled: Count of cancelled flights 
- Arr_Diverted: Count of aircrafts diverted 
- Arr_Delay: Total minutes delayed 
- Carrier_Delay: Total minutes delayed due to the carrier  
- Weather_Delay: Total minutes delayed due to weather
- NAS_Delay: Total minutes delayed due to National Aviation System
- Security_Delay: Total minutes delayed due to Security 
- Late_Aircraft_Delay: Total minutes delayed due to late arriving aircraft 


** Please note all "count" columns are pro-rated by minutes. Do if the delays was 45 minutes and 15 minutes was due to weather and 30 minutes was due to Late Aircraft the count would be .33 for weather and .66 for Late Aircraft. 


##### How are these categories defined?

##### Air Carrier: The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc.).

##### Extreme Weather: Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.

##### National Aviation System (NAS): Delays and cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.

##### Late-arriving aircraft: A previous flight with same aircraft arrived late, causing the present flight to depart late.

##### Security: Delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.

## Analysis  1 
Our end goal is to get an understanding of how Airline delays have changed over time. We start our analysis with Query 1, comparing total delayed flights by reason (carrier, weather, nas, security, and late aircraft).  

### Causes of Delays Over Peak Covid 

In [ ]:

q = """
SELECT year,
SUM(CAST(carrier_ct AS FLOAT)) as Carrier_Delay,
SUM(CAST(weather_ct AS FLOAT)) as Weather_Delay,
SUM(CAST(nas_ct AS FLOAT)) as NAS_Delay,
SUM(CAST(security_ct AS FLOAT)) as Security_Delay,
SUM(CAST(late_aircraft_ct AS FLOAT)) as Late_Aircraft_Delay
FROM AIRLINE_DELAY_CAUSE_CURRENT
GROUP BY year ORDER BY YEAR"""

pd.options.display.float_format = '{:,.2f}'.format
delays_cause_df = pd.read_sql_query(q, nzcon)
#delays_cause_df.columns = delays_cause_df.columns.str.decode('utf-8')
delays_cause_df

The following code is to display a bar shart visualizing the different delay counts by reason. This way we can quickly assess why flights may have been having issues. 

In [ ]:


# set width of bars
barWidth = 0.15

# Set position of bar on X axis
r1 = np.arange(len(delays_cause_df['CARRIER_DELAY']))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
r5 = [x + barWidth for x in r4]

plt.figure(figsize=(15, 6))  # width:10, height:8

# Make the plot
plt.bar(r1, delays_cause_df['CARRIER_DELAY'], color='blue', width=barWidth, edgecolor='white', label='Carrier Delay')
plt.bar(r2, delays_cause_df['WEATHER_DELAY'], color='orange', width=barWidth, edgecolor='white', label='Weather Delay')
plt.bar(r3, delays_cause_df['NAS_DELAY'], color='pink', width=barWidth, edgecolor='white', label='NAS Delay')
plt.bar(r4, delays_cause_df['LATE_AIRCRAFT_DELAY'], color='purple', width=barWidth, edgecolor='white', label='Late Aircraft Delay')
plt.bar(r5, delays_cause_df['SECURITY_DELAY'], color='green', width=barWidth, edgecolor='white', label='Security Delay')

# Add xticks on the middle of the group bars
plt.xlabel('Year', fontweight='bold')
plt.ylabel('Total Flights', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(delays_cause_df['CARRIER_DELAY']))], ['2019', '2020', '2021', '2022'])



# Create legend & Show graphic
plt.legend()
plt.show()


With this data now visualized, it is clear there are some major descrepancies both by year as well as reason. It seems like security days are minumal accross all years. While Carrier and Late Aircrafts are the number 1 and 2 cause for delays consistently. An uninformed viewer may interpret 2020 and 2022 as being a particularly great years to travel since there were significantly less delays. However, we understand that beacause of Covid this may be skewed. Additionally we have not completed 2022 so there is not a full year of data to compare over time. 

This prompts us into our next query to better compare this data over time by normalizing it. Taking the total amount of delays over the total number of flights. By doing this, we will be able to make deductions more fairly. 

## Analysis 2

The following code cells are a build to the final graph. 
First we collect sums of Total Cancelled Flights, Total Delayed Flights, Total Flights, Carrier Delays, Weather Delays, NAS Delays, Security Delays and Late Aircraft Delays. 
Next, using the pandas and numpy python libraries, we can operate on our dataframe to get ratios for Delayed, Cancelled, and total Distrupted Flights (Delayed and Canceled combined). 
Lastly, we can create a new data frame from our original with percentage of delayed flights that had to do with Carrier, Weather, NAS, Security, and Late Aircraft Delays. 
The final cell combines all this data into a normalized and informed graph that displays distrupted flight ratios between 2019 and 2022.

### Understanding Delays with Context

In [ ]:

q = """
SELECT year,
SUM(CAST(arr_cancelled AS FLOAT)) as Total_Cancelled_Flights,
SUM(CAST(arr_del15 AS FLOAT)) as Total_Delayed_Flights,
SUM(CAST(arr_flights AS FLOAT)) as Total_Flights,
SUM(CAST(carrier_ct AS FLOAT)) as Carrier_Delay,
SUM(CAST(weather_ct AS FLOAT)) as Weather_Delay,
SUM(CAST(nas_ct AS FLOAT)) as NAS_Delay,
SUM(CAST(security_ct AS FLOAT)) as Security_Delay,
SUM(CAST(late_aircraft_ct AS FLOAT)) as Late_Aircraft_Delay
FROM AIRLINE_DELAY_CAUSE_CURRENT
GROUP BY year ORDER BY YEAR"""

pd.options.display.float_format = '{:,.2f}'.format
delays_cause_df2 = pd.read_sql_query(q, nzcon)
#delays_cause_df2.columns = delays_cause_df2.columns.str.decode('utf-8')
delays_cause_df2

#### Pandas Operations

In [ ]:
delays_cause_df2['Delayed:Total Ratio'] = 100*(delays_cause_df2['TOTAL_DELAYED_FLIGHTS']/delays_cause_df2['TOTAL_FLIGHTS'])
delays_cause_df2

In [ ]:
delays_cause_df2['Cancelled:Total Ratio'] = 100*(delays_cause_df2['TOTAL_CANCELLED_FLIGHTS']/delays_cause_df2['TOTAL_FLIGHTS'])
delays_cause_df2

In [ ]:
delays_cause_df2['Distrupted:Total Ratio'] = ((delays_cause_df2['TOTAL_CANCELLED_FLIGHTS']+delays_cause_df2['TOTAL_DELAYED_FLIGHTS'])/delays_cause_df2['TOTAL_FLIGHTS'])*100
delays_cause_df2


In [ ]:
delays_cause_df2_new = delays_cause_df2[['YEAR','CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'TOTAL_DELAYED_FLIGHTS','Delayed:Total Ratio']].copy()
delays_cause_df2_new['CARRIER_DELAY'] = (delays_cause_df2_new['CARRIER_DELAY']/delays_cause_df2_new['TOTAL_DELAYED_FLIGHTS'])*delays_cause_df2_new['Delayed:Total Ratio']
delays_cause_df2_new['WEATHER_DELAY'] = (delays_cause_df2_new['WEATHER_DELAY']/delays_cause_df2_new['TOTAL_DELAYED_FLIGHTS'])*delays_cause_df2_new['Delayed:Total Ratio']
delays_cause_df2_new['NAS_DELAY'] = (delays_cause_df2_new['NAS_DELAY']/delays_cause_df2_new['TOTAL_DELAYED_FLIGHTS'])*delays_cause_df2_new['Delayed:Total Ratio']
delays_cause_df2_new['SECURITY_DELAY'] = (delays_cause_df2_new['SECURITY_DELAY']/delays_cause_df2_new['TOTAL_DELAYED_FLIGHTS'])*delays_cause_df2_new['Delayed:Total Ratio']
delays_cause_df2_new['LATE_AIRCRAFT_DELAY'] = (delays_cause_df2_new['LATE_AIRCRAFT_DELAY']/delays_cause_df2_new['TOTAL_DELAYED_FLIGHTS'])*delays_cause_df2_new['Delayed:Total Ratio']

delays_cause_df2_new = delays_cause_df2_new.drop(columns=['TOTAL_DELAYED_FLIGHTS', 'Delayed:Total Ratio'])
delays_cause_df2_new



In [ ]:

# set width of bars
barWidth = 0.15

# Set position of bar on X axis
r1 = np.arange(len(delays_cause_df2['YEAR']))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

plt.figure(figsize=(14, 6))  # width:10, height:8

# Make the plot
#plt.bar(r1, delays_cause_df2['Delayed:Total Ratio'], color='#d0c4ef', width=barWidth, edgecolor='white', label='Delay Ratio')
plt.bar(r2, delays_cause_df2['Cancelled:Total Ratio'], color='#c75f44', width=barWidth, edgecolor='white', label='Cancelled Ratio')
plt.bar(r3, delays_cause_df2['Distrupted:Total Ratio'], color='#7e9b89', width=barWidth, edgecolor='white', label='Distrupted Ratio')
plt.bar(r1, delays_cause_df2_new['CARRIER_DELAY'], color='#d0c4ef', width=barWidth, edgecolor='white', label='CARRIER_DELAY')
plt.bar(r1, delays_cause_df2_new['WEATHER_DELAY'], bottom=delays_cause_df2_new['CARRIER_DELAY'], color='#7e9bb9', width=barWidth, edgecolor='white', label='WEATHER_DELAY')
plt.bar(r1, delays_cause_df2_new['NAS_DELAY'], bottom=delays_cause_df2_new['CARRIER_DELAY']+delays_cause_df2_new['WEATHER_DELAY'], color='#4e9bc9', width=barWidth, edgecolor='white', label='NAS_DELAY')
plt.bar(r1, delays_cause_df2_new['SECURITY_DELAY'], bottom=delays_cause_df2_new['CARRIER_DELAY']+delays_cause_df2_new['WEATHER_DELAY']+delays_cause_df2_new['NAS_DELAY'], color='#2e9b89', width=barWidth, label='SECURITY_DELAY')
plt.bar(r1, delays_cause_df2_new['LATE_AIRCRAFT_DELAY'], bottom=delays_cause_df2_new['CARRIER_DELAY']+delays_cause_df2_new['WEATHER_DELAY']+delays_cause_df2_new['NAS_DELAY']+delays_cause_df2_new['SECURITY_DELAY'], color='#3eab89', width=barWidth, edgecolor='white', label='LATE_AIRCRAFT_DELAY')




# Add xticks on the middle of the group bars
plt.xlabel('Year', fontweight='bold')
plt.ylabel('Percentage', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(delays_cause_df2['YEAR']))], ['2019', '2020', '2021', '2022'])


plt.title(label = "Normalized Distruption Ratios")
# Create legend & Show graphic
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0)
plt.show()


With this new normalized view of our data, we can see that our initial thoughts were not completely fair. 2020 was not a great year to travel due to cancelations, however of the flights that did operate, the overal chance of delays was relatively low compared to the other years. 2022 looked as though it had low delay quantity but when analyzing the amount of delays over the total, we can clearly see 2022 has not been performing well so far with the highest change of distrupted flights overall. 

This gives up a very strong understanding of how the airline industry has been performing overall for the past couple of years. We want to continue our analysis to include historical data and determine if these trends have been common in the past or have been improving over time. 

## Analysis 3 

Its good practice to get an overall understanding of your data before diving deeper. The first query run with our historical data is total delays by year between 2003 and 2018. 

### Comparing Current to Historical 

In [ ]:
q = """
SELECT 
year, SUM(CAST(arr_del15 as float)) as "Total Delays"
FROM AIRLINE_DELAY_CAUSE_HISTORY GROUP BY year order by year asc
"""

all_delay_df = pd.read_sql_query(q, nzcon)
#all_delay_df.columns = all_delay_df.columns.str.decode('utf-8')
all_delay_df

In [ ]:
plt.plot(all_delay_df["YEAR"],all_delay_df["Total Delays"])
plt.title('Total Delays by Year')
plt.xlabel('Year')
plt.ylabel('Total Delays')
plt.rcParams["figure.figsize"] = (17,5)
plt.show()


A quick and easy line graph shows overall trend has been a decline in delays. Lets see if the rest of our data support this. 

## Analysis 4 

The goal of analysis query 4 is to get the total average delays by month between 2003 and 2012 and compare that to the total avereage delays by month between 2013-2018. This can answer the question, "Overall how were airlines operating in the eary 2000s compared to how they were operating in the later 2010s?" 


First we run a query to ensure we have gathered the total delays per month by year. 

### Total Delays by Month and Year

In [ ]:
q = """
SELECT 
year, month,
SUM(arr_del15) as TOTAL_DELAYS_PER_MONTH
FROM AIRLINE_DELAY_CAUSE_HISTORY GROUP BY year, month order by year, month
"""

delay_by_month_df1 = pd.read_sql_query(q, nzcon)
#delay_by_month_df1.columns = delay_by_month_df1.columns.str.decode('utf-8')

delay_by_month_df1

From this data we are able to take the avereage by month between 2003 and 2012 as well as the avereage by month between 2013 and 2018. 

### Average Delays by Month from 2003-2012 Compared to 2013-2018

In [ ]:


q = """
SELECT t1.FINAL_MONTH1 as Month, t1."2003-2012", t2."2013-2018"

FROM 

(SELECT month as FINAL_MONTH1, AVG(SUM1) as "2003-2012" 
FROM
(SELECT 
month, year,
SUM(CAST(arr_del15 as FLOAT)) as SUM1
FROM AIRLINE_DELAY_CAUSE_HISTORY 
GROUP BY year, month order by year, month) as t02
WHERE year <= 2012 GROUP BY FINAL_MONTH1 ORDER BY FINAL_MONTH1) as t1

INNER JOIN 

(SELECT month as FINAL_MONTH2, AVG(SUM2) as "2013-2018" 
FROM
(SELECT 
month, year,
SUM(CAST(arr_del15 as FLOAT)) as SUM2
FROM AIRLINE_DELAY_CAUSE_HISTORY
GROUP BY year, month order by year, month) as t02
WHERE year >= 2013 and year <= 2019 GROUP BY FINAL_MONTH2 ORDER BY FINAL_MONTH2) as t2

ON t1.FINAL_MONTH1=t2.FINAL_MONTH2 ORDER BY t1.FINAL_MONTH1

"""
pd.options.display.max_rows = None
delay_by_month_df2 = pd.read_sql_query(q, nzcon)
#delay_by_month_df2.columns = delay_by_month_df2.columns.str.decode('utf-8')

delay_by_month_df2


At a glance, our table is a nice comparison but lets try to run some additional analysis and view it in a graph to get a better understanding. 

In [ ]:
array_1 = delay_by_month_df2['2003-2012']
array_2 = delay_by_month_df2['2013-2018']

sum_total = 0

zip_object = zip(array_1, array_2)
for element1, element2 in zip_object:
    if (element1 > element2):
        sum_total += 1


early_sum = delay_by_month_df2['2003-2012'].sum()
late_sum = delay_by_month_df2['2013-2018'].sum()
print(str(sum_total) + " out of 12 months the early 2000s had more delays than late 2000s")
print(((late_sum - early_sum)/late_sum)*100)

Above, we have compared each month and determined that every month between 2003-2012 on avereage had more delays than the some month between 2012 and 2018. In summary, this is a 20% decline in delays between each span of years. This confirms our initial analysis that airlines delays were in decline leading up to 2019. 

### Delays Compared 2003-2012 and 2013-2019

In [ ]:

# set width of bars
barWidth = 0.25

# Set position of bar on X axis
r1 = np.arange(len(delay_by_month_df2['MONTH']))
r2 = [x + barWidth for x in r1]

plt.figure(figsize=(15, 6))  # width:10, height:8

# Make the plot
plt.bar(r1, delay_by_month_df2['2003-2012'], color='blue', width=barWidth, edgecolor='white', label='Delays 2003-2012')
plt.bar(r2, delay_by_month_df2['2013-2018'], color='green', width=barWidth, edgecolor='white', label='Delays 2013-2018')

# Add xticks on the middle of the group bars
plt.xlabel('Month', fontweight='bold')
plt.ylabel('Total Delays', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(delay_by_month_df2['MONTH']))], delay_by_month_df2['MONTH'])



# Create legend & Show graphic
plt.legend()
plt.show()


Data visualization with a bar chart confirms this. 

## Analysis 5 

Now that we have seen a decline in flight delays leading up to 2019, lets compare to our current data and answer the question, "Are flight delays still in decline or have they risen since the impact of COVID-19?" 

We run a simple query to see how many delays we have had each month in 2022 

### Delay Trends for 2022

In [ ]:
q = """
SELECT month,

SUM(case when year = 2022 then SUM else 0 end) as "2022" FROM 

(SELECT 
month, year,
SUM(CAST(arr_del15 as FLOAT)) as SUM
FROM AIRLINE_DELAY_CAUSE_CURRENT GROUP BY year, month order by year, month) as t1 

GROUP BY month order by month
"""

delay_by_month_df3 = pd.read_sql_query(q, nzcon)
#delay_by_month_df3.columns = delay_by_month_df3.columns.str.decode('utf-8')
delay_by_month_df3

### Total Compared Delays over Time 

Putting that data together with the original historical data we see ...

In [ ]:

# set width of bars
barWidth = 0.25

# Set position of bar on X axis
r1 = np.arange(len(delay_by_month_df2['MONTH']))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

plt.figure(figsize=(15, 6))  # width:10, height:8

# Make the plot
plt.bar(r1, delay_by_month_df2['2003-2012'], color='#4169e1', width=barWidth, edgecolor='white', label='Delays 2003-2012')
plt.bar(r2, delay_by_month_df2['2013-2018'], color='#93c47d', width=barWidth, edgecolor='white', label='Delays 2013-2018')
plt.bar(r3, delay_by_month_df3['2022'], color='#ca4c4c', width=barWidth, edgecolor='white', label='Delays 2022')

# Add xticks on the middle of the group bars
plt.xlabel('Month', fontweight='bold')
plt.ylabel('Total Delays', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(delay_by_month_df2['MONTH']))], delay_by_month_df2['MONTH'])



# Create legend & Show graphic
plt.legend()
plt.show()

As determined earlier, 2022 has been trending very poorly for delays when compared to 2019, 2020 and 2021. We can see here in this graph that it is also trending very poorly against the averages in the late 2010s when delays were improving and in some cases is even worse than the avereage in the earlier 2000s. This is a fantastic visual to understand how the airline industry has not recovered effectively after the COVID-19 pandemic or has been impacted by other outside facters causing worsening delays. 

## Analysis 6 

For the end of our analysis we'd like to understand which airline carriers and airports are best to use if a passenger is trying to avoid delays. Since many of the airlines have merged or no longer exist, we'd only like to suggest currently operating carriers as well as major airports with over 100,000 arriving flights overall. 

### Delays by Carrier 

In [ ]:
q = """
SELECT carrier_name, sum(arr_del15)

FROM AIRLINE_DELAY_CAUSE_HISTORY GROUP BY carrier_name ORDER BY carrier_name

"""

delay_by_month_df3 = pd.read_sql_query(q, nzcon)
#delay_by_month_df3.columns = delay_by_month_df3.columns.str.decode('utf-8')
delay_by_month_df3

From our current collection of data we would like to get a list of all active carriers. 

In [ ]:
## find Carriers that are still operating 

q = """
SELECT UNIQUE(carrier_name)
FROM AIRLINE_DELAY_CAUSE_CURRENT

"""
carrier_names_df = pd.read_sql_query(q, nzcon)
#carrier_names_df.columns = carrier_names_df.columns.str.decode('utf-8')
carrier_list = carrier_names_df['CARRIER_NAME'].to_list()
carriers = "','".join(carrier_list)
carrier_list


Using this list we will make sure to include only data from these carriers and sumarize total delays over total flights. 

In [ ]:
q = """
SELECT carrier_name
FROM AIRLINE_DELAY_CAUSE_CURRENT where carrier_name IN ('{}')

""".format(carriers)

carrier_names_df = pd.read_sql_query(q, nzcon)
#carrier_names_df.columns = carrier_names_df.columns.str.decode('utf-8')
carrier_names_df


In [ ]:
q = """
SELECT 
t1.carrier_name,
SUM(t1.TOTAL_FLIGHTS) as TOTAL_FLIGHTS,
SUM(t1.Total_Delayed_Flights) as Total_Delayed_Flights,
SUM(t1.Diverted_Flights) as Diverted_Flights,
SUM(t1.Cancelled_Flights) as Cancelled_Flights,
(SUM(t1.Diverted_Flights) + SUM(t1.Total_Delayed_Flights) + SUM(t1.Cancelled_Flights))/SUM(t1.TOTAL_FLIGHTS) as Delay_Percentage

FROM 
(SELECT carrier_name,
SUM(CAST(arr_flights as FLOAT)) as TOTAL_FLIGHTS,
SUM(CAST(arr_del15 as FLOAT)) as Total_Delayed_Flights,
SUM(CAST(arr_diverted as FLOAT)) as Diverted_Flights,
SUM(CAST(arr_cancelled as FLOAT)) as Cancelled_Flights
FROM 
AIRLINE_DELAY_CAUSE_CURRENT GROUP BY AIRLINE_DELAY_CAUSE_CURRENT.CARRIER_NAME

UNION ALL

SELECT carrier_name,
SUM(CAST(arr_flights as FLOAT)) as TOTAL_FLIGHTS,
SUM(CAST(arr_del15 as FLOAT)) as Total_Delayed_Flights,
SUM(CAST(arr_diverted as FLOAT)) as Diverted_Flights,
SUM(CAST(arr_cancelled as FLOAT)) as Cancelled_Flights
FROM 
AIRLINE_DELAY_CAUSE_HISTORY

GROUP BY AIRLINE_DELAY_CAUSE_HISTORY.CARRIER_NAME
) as t1

GROUP BY t1.carrier_name ORDER BY Delay_Percentage ASC
"""


carrier_delay_df = pd.read_sql_query(q, nzcon)
#carrier_delay_df.columns = carrier_delay_df.columns.str.decode('utf-8')
carrier_delay_df

Of the currently operating flights we would most commonly recomend flying with Delta 

We will run a similar query for Airports. 

### Airport Delays 

In [ ]:


q = """
SELECT 
t1.airport_name,
SUM(t1.TOTAL_FLIGHTS) as TOTAL_FLIGHTS,
SUM(t1.Total_Delayed_Flights) as Total_Delayed_Flights,
SUM(t1.Diverted_Flights) as Diverted_Flights,
SUM(t1.Cancelled_Flights) as Cancelled_Flights,
(SUM(t1.Diverted_Flights) + SUM(t1.Total_Delayed_Flights) + SUM(t1.Cancelled_Flights))/SUM(t1.TOTAL_FLIGHTS) as Distrupted_Percentage

FROM 
(SELECT airport_name,
SUM(CAST(arr_flights as FLOAT)) as TOTAL_FLIGHTS,
SUM(CAST(arr_del15 as FLOAT)) as Total_Delayed_Flights,
SUM(CAST(arr_diverted as FLOAT)) as Diverted_Flights,
SUM(CAST(arr_cancelled as FLOAT)) as Cancelled_Flights
FROM 
AIRLINE_DELAY_CAUSE_CURRENT GROUP BY AIRLINE_DELAY_CAUSE_CURRENT.airport_name

UNION ALL

SELECT airport_name,
SUM(CAST(arr_flights as FLOAT)) as TOTAL_FLIGHTS,
SUM(CAST(arr_del15 as FLOAT)) as Total_Delayed_Flights,
SUM(CAST(arr_diverted as FLOAT)) as Diverted_Flights,
SUM(CAST(arr_cancelled as FLOAT)) as Cancelled_Flights
FROM 
AIRLINE_DELAY_CAUSE_HISTORY WHERE arr_flights > 10000

GROUP BY AIRLINE_DELAY_CAUSE_HISTORY.airport_name
) as t1 WHERE t1.TOTAL_FLIGHTS > 100000

GROUP BY t1.airport_name ORDER BY Distrupted_Percentage DESC
"""

airport_delay_df = pd.read_sql_query(q, nzcon)
#airport_delay_df.columns = airport_delay_df.columns.str.decode('utf-8')
airport_delay_df



Of the major airports operating 100,000 flights or more, we would suggest to avoid Newark, LaGuardia, Fort Lauderdale and Orlando. 

## Conclusion 

We have conluded out data analysis for Airline Flight Delay data. To review we first looked at current delay data between 2019 and 2022. Because we are informed with knowledge of the COVID-19 pandemic as a possible distruptor to this data we normalized the information in order to compare distrupted flight ratios across all three years and found that 2022 seemed to be trending with an increase in delays. 
Next, there were several queries to compare delay trends between 2003 and 2012 to the years leading up to 2019 from 2013-2018. By taking the avereage delays per month, the analysis showed that flight delays were descreasing in the late 2010s. 
While we are still recovering from the covid-19 pandemic and are impacted by many other global factors, when looking at 2022 compared to pre pandemic delay trends, we see that there is a definite increase in delays. 
Lastly, this data included information about Airports and Carriers that were operating flights between 2003 and 2022. The final queries are able to tell which Carriers operate the least amount of delays and which Aiports have the most delays. This helps give a recomendation to travelers on which carriers to fly with and from where. 



In [ ]:
q = """
select count(*) from demo.EXT_AIRLINE_DELAY_CAUSE_2018
"""

start_time = datetime.datetime.now()
df = pd.read_sql_query(q, nzcon)
end_time = datetime.datetime.now()
runtime = end_time - start_time
#df.columns = df.columns.str
print(df)
print(f"Query Elapsed Time: {runtime}")

In [ ]:
q = """
select count(*) from demo.AIRLINE_DELAY_CAUSE_2018_LOCAL
"""
start_time = datetime.datetime.now()
df = pd.read_sql_query(q, nzcon)
end_time = datetime.datetime.now()
runtime = end_time - start_time
#df.columns = df.columns.str
print(df)
print(f"Query Elapsed Time: {runtime}")